#Using ```tweepy``` for Data Mining

This is an introduction to data collection from <a href="http://www.twitter.com/">Twitter</a> using the ```tweepy``` package.

In [36]:
### DS 710: final project: collect and analyze twitter data
### Connie Sosa: 11/30/2015

# You must install the tweepy package before using it!
import twitter
import tweepy
import io
import json

You'll need to set up an app at <a href="https://apps.twitter.com/">apps.twitter.com</a>.

In [37]:
# Save your consumer key, consumer secret, access token, and access secret here.
# Don't share these secrets with others!  If you're building code for Twitter which many people will share,
# you should encrypt this information.  
# It's also possible to generate access tokens and secrets from within an app.

con_key = 'bdCAhjLwnAguqzNqVr2ucHN5Q'
con_secret = '7z65BYKQpJIRA1tuE05XUb8MwjstV5iLHMAITxUMV54ekBHZMT'
acc_token = '4293166817-xzOF6TKYNVRyNvy3fpN9MhAtNUdDZibxx3SFZeV'
acc_secret = '74ZX10bhpr4KPHFQVbDDKQ7onU4fbVUPmw5s4SBMwntTJ'

In [17]:
###############################################################################
#
# get_tweepy_api(): Return authenticated twitter service api
#
###############################################################################
def get_tweepy_api():
    #Use tweepy.OAuthHandler to create an authentication using the given key and secret
    auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
    auth.set_access_token(acc_token, acc_secret)

    #Connect to the Twitter API using the authentication
    api = tweepy.API(auth)
    return api


In [59]:
###############################################################################
#
# get_twitter_api(): Return authenticated twitter service api
#
###############################################################################
def get_twitter_api():

    auth = twitter.oauth.OAuth(acc_token, acc_secret, con_key, con_secret)
    twitter_api = twitter.Twitter(auth=auth)
 
    return twitter_api

In [64]:
tweet_list = api.search(q='hamradio')
len(tweet_list)

def print_tweet(tweet):
    print(tweet.text)
    
tweet=tweet_list[1]
print_tweet(tweet)

HF Band Day/Night . 80m-40m  Poor/Fair . 30m-20m  Fair/Fair . 17m-15m  Fair/Fair . 12m-10m  Poor/Poor #hamradio #hamr


In [65]:
###############################################################################
#
# twitter_search(): Search twitter in batches. From [MINING the Social Web by M. Russell]
#
###############################################################################
def twitter_search(twitter_api, q, max_results=200, **kw):

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets and 
    # https://dev.twitter.com/docs/using-search for details on advanced 
    # search criteria that may be useful for keyword arguments

    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
#    search_results = twitter_api.search(q=q, count=100, **kw)
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://dev.twitter.com/docs/rate-limiting/1.1/limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
#        except e: # No more results when next_results doesn't exist
#            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                for kv in next_results[1:].split("&") ])
        
#        search_results = twitter_api.search.tweets(**kwargs)
        search_results = twitter_api.search(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses


## The REST API

Twitter has two versions of its API.

The REST API allows you to _pull_ information from Twitter, or _push_ information back to Twitter.  We'll use the REST API to run a specific search.  You could also use the REST API to make automatic tweets on Twitter, or get information about specific users.

In [29]:
###############################################################################
#
# save_json(): Save to JSON. From [MINING].
#
###############################################################################
def save_json(filename, data):
    print ("In save_json()")
    with io.open('storage/{0}.json'.format(filename), 
                 'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(data, ensure_ascii=False)))


In [30]:
###############################################################################
#
# load_json(): Save to JSON.  From [MINING].
#
###############################################################################
def load_json(filename):
    print ("In load_json()")
    with io.open('storage/{0}.json'.format(filename), 
                 encoding='utf-8') as f:
        return f.read()


In [31]:
###############################################################################
#
# call_tweepy_search_sample()
#
###############################################################################
def call_tweepy_search_sample():
    api = get_tweepy_api()

    tweet_list = api.search(q='#%23hamradio')
    tweet_list[0]
    dir(tweet_list[0])

#    for tweet in tweet_list:
#       print tweet.text
    [tweet.text for tweet in tweet_list]


In [52]:
###############################################################################
#
# call_simple_twitter_search_sample()
#
###############################################################################
def call_simple_twitter_search_sample():
    api = get_twitter_api()

    tweet_list = api.search(q='bigdata')
    tweet_list[0]
    dir(tweet_list[0])

    for tweet in tweet_list:
       print( tweet.text)

We retrieve a SearchResult object for each tweet, full of data such as the language, the identity of the poster, etc.

In [50]:
###############################################################################
# call_twitter_search_sample()
###############################################################################
def call_twitter_search_sample():
    twitter_service_api = get_twitter_api()
    q='#%23hamradio'
    results = twitter_search(twitter_service_api, q, max_results=100)
        
    ## Show one sample search result by slicing the list...
    for i in range(100):
        print (json.dumps(results[i], indent=1))

In [66]:
call_simple_twitter_search_sample()

TwitterHTTPError: Twitter sent status 404 for URL: 1.1/search.json using parameters: (oauth_consumer_key=bdCAhjLwnAguqzNqVr2ucHN5Q&oauth_nonce=2571453778550442222&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1449546054&oauth_token=4293166817-xzOF6TKYNVRyNvy3fpN9MhAtNUdDZibxx3SFZeV&oauth_version=1.0&q=bigdata&oauth_signature=qvkRvk3t72xUJcltmNidy5cHbFI%3D)
details: {'errors': [{'code': 34, 'message': 'Sorry, that page does not exist'}]}

In [33]:
def print_tweet(tweet):
    print(tweet.text)
    

In [45]:
##########################################
#### try to save it to json files but with all escapes.
##########################################
# Step 1
twitter_service_api = get_twitter_api()

#auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
#auth.set_access_token(acc_token, acc_secret)
#Connect to the Twitter API using the authentication
#api = tweepy.API(auth)
    
q = "hamradio"

# Step 2
results = twitter_search(twitter_service_api, q, max_results=1000)

print ("before dump json results")
print (json.dumps(results, indent=1))

##for i in range(100):
 ##   print json.dumps(results[i], indent=1)

# Step 3
print("before file write")
save_json(q, results)

# Step 4
print("before load_json")
read_results=load_json(q)

print ("before dump json read_results")
print (json.dumps(read_results, indent=1))

##for i in range(100):
 ##   print json.dumps(read_results[i], indent=1)



TwitterHTTPError: Twitter sent status 404 for URL: 1.1/search.json using parameters: (count=100&oauth_consumer_key=bdCAhjLwnAguqzNqVr2ucHN5Q&oauth_nonce=725011974824766513&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1449543999&oauth_token=4293166817-xzOF6TKYNVRyNvy3fpN9MhAtNUdDZibxx3SFZeV&oauth_version=1.0&q=hamradio&oauth_signature=etYSyHvUFIsU69zRdj8b%2BqJhBoE%3D)
details: {'errors': [{'code': 34, 'message': 'Sorry, that page does not exist'}]}

In [12]:
userInfo = tweet.author

NameError: name 'tweet' is not defined

In [21]:
tweet_list[0]

NameError: name 'tweet_list' is not defined

In [30]:
tweet_list[1]
print (json.dumps(results, indent=1))


NameError: name 'tweet_list' is not defined

In [23]:
#We can use the dir command to view a list of the attributes of each tweet

dir(tweet_list[0])

NameError: name 'tweet_list' is not defined

In [24]:
#Let's display the text of each tweet we found.
[tweet.test for tweet in tweet_list]

NameError: name 'tweet_list' is not defined

In [25]:
[tweet.id for tweet in tweet_list]   ### id of the tweet

NameError: name 'tweet_list' is not defined

NameError: name 'tweet_list' is not defined

In [27]:
[tweet.user for tweet in tweet_list]

NameError: name 'tweet_list' is not defined

## The Streaming API

The Streaming API allows us to monitor Twitter in real time, grabbing tweets as they are made.

The ```tweepy``` package includes a class called ```StreamListener``` which monitors Twitter for us.  However, by default StreamListener does nothing with the tweets it collects.

In this demonstration, we'll modify ```StreamListener``` to make a class that prints each tweet we're interested in to the screen.  Later, you may wish to create your own class which saves information from tweets to a file.

In [21]:
#We create a subclass of tweepy.StreamListener to add a response to on_status

class PrintingStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        print(status.text)
        
    #disconnect the stream if we receive an error message indicating we are overloading Twitter
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
 

Once we have created our subclass, we can set up our own Twitter stream.

In [ ]:
### code from sentdex saving Tweets 2 of 3 and cleaning up tweets: how to use the twitter API v 1.1 w
### with python to stream tweets 3 of 3.

class PrintingStreamListener(tweepy.StreamListener):

    def on_data(self, data):
        try:
            print data
            
            tweetData = data.split(',"')
            saveFile = open('twitDB.csv', 'a')
            saveFile.write(data)
            saveFile.write('\n')
            saveFile.close()
            return True
        except BaseException, e:
            print 'failed ondata,', str(e)
            time.sleep(5)
        
    #disconnect the stream if we receive an error message indicating we are overloading Twitter
    
    def on_error(self, status):
        print status
        return False
 

In [22]:
#We create and authenticate an instance of our new ```PrintingStreamListener``` class

my_stream_listener = PrintingStreamListener()
my_stream = tweepy.Stream(auth = api.auth, listener=my_stream_listener)

We'll use the ```track``` command to look for tweets with a specific keyword.  You can read more about constructing searches with ```track``` in the <a href="https://dev.twitter.com/streaming/overview/request-parameters#track">Twitter API documentation</a>.

In [23]:
# Now, we're ready to start streaming!  We'll look for recent tweets which use the word "data".
# You can pause the display of tweets by interrupting the Python kernel.

my_stream.filter(track=['data'])

RT @KardashianReact: When a girl says "first of all" run away bc that means that she's prepared research, data, &amp; charts on your fuckup. Sh…
The fiddling with temperature data is the biggest science scandal ever | via @Telegraph https://t.co/N57Z5WsalY
95% off Learning Python for Data Analysis and Visualization =&gt; https://t.co/oVT6u1HNPk ... #udemy #dataviz https://t.co/7WNrfvMnrr
Better question: Have we explained why we want their data and how we could use it to better their experience.  #ascilite2015 2/2
RT @wattsnextBen: OrganisationView #workometry @AndrewMarritt #HRTF2015
#data https://t.co/ENDHDauXGi
#admin #jobs Travel Data Analyst: Day to day admin and reporting of weekly sales figures to Manage... https://t.co/M9EjIqnouT #Colchester
Hillary Clinton, Bernie Sanders support marijuana reform, citing faulty data - Washington Times https://t.co/BfbIKV7U7f
@LuisMellet Hola, perdi toda mi data, estoy en el terruño, por mensaje privado enviame tus telfs. Abrazos, Vivian
RT @wh

KeyboardInterrupt: 

In [24]:
# Even if you pause the display of tweets, your stream is still connected to Twitter!
# To disconnect (for example, if you want to change which words you are searching for), 
# use the disconnect() function.

my_stream.disconnect()